# NLSQ Interactive Tutorial: GPU-Accelerated Curve Fitting

**Welcome to NLSQ!** This interactive tutorial will guide you through using NLSQ for fast, GPU-accelerated curve fitting.

## What You'll Learn

1. ✅ Installation and setup (CPU and GPU)
2. 📈 Basic curve fitting with common models
3. 🎯 Parameter bounds and constraints
4. 🔧 Error handling and diagnostics
5. 💾 Large dataset handling (millions of points)
6. ⚡ GPU acceleration (100x+ speedups)
7. 🚀 Advanced features (callbacks, robust fitting, auto p0)

**Time**: ~45 minutes  
**Level**: Beginner to Intermediate  
**Prerequisites**: Basic Python, NumPy

---

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/imewei/NLSQ/blob/main/examples/NLSQ_Interactive_Tutorial.ipynb)

## Section 1: Installation & Setup

### 1.1 Installation

NLSQ requires JAX for GPU acceleration. On Google Colab, JAX is pre-installed with GPU support.

**Installation options**:
- **Colab (GPU)**: JAX pre-installed ✅
- **Local (CPU)**: `pip install nlsq`
- **Local (GPU)**: Install JAX with GPU support first, then `pip install nlsq`

In [1]:
# Install NLSQ (skip if already installed)
!pip install -q nlsq

# Check installation
import nlsq

print(f"NLSQ version: {nlsq.__version__}")
print("✅ Installation successful!")

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.12/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai

INFO:2025-11-17 16:32:28,188:jax._src.xla_bridge:808: Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory


Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory


NLSQ version: 0.2.1.post25
✅ Installation successful!


### 1.2 Imports

Let's import the libraries we'll need:

In [2]:
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np

from nlsq import (
    callbacks,  # Progress monitoring
    curve_fit,
    functions,  # Common fitting functions
)

# Set random seed for reproducibility
np.random.seed(42)

print("✅ All imports successful!")

✅ All imports successful!


### 1.3 Check GPU Availability

NLSQ automatically uses GPU if available. Let's check:

In [3]:
import jax

# Check available devices
devices = jax.devices()
print(f"Available devices: {devices}")
print(f"Default backend: {devices[0].platform}")

if devices[0].platform == "gpu":
    print("\n🚀 GPU detected! NLSQ will use GPU acceleration.")
else:
    print("\n💻 Running on CPU. For GPU, use Runtime -> Change runtime type -> GPU")

Available devices: [CudaDevice(id=0)]
Default backend: gpu

🚀 GPU detected! NLSQ will use GPU acceleration.


---

## Section 2: Your First Curve Fit

Let's start with a simple example: fitting an exponential decay curve.

### 2.1 Generate Sample Data

We'll create noisy data following an exponential decay: $y = a \cdot e^{-b \cdot x} + c$

In [4]:
# True parameters
a_true, b_true, c_true = 10.0, 0.5, 2.0

# Generate x data
x = np.linspace(0, 10, 100)

# Generate y data with noise
y_true = a_true * np.exp(-b_true * x) + c_true
noise = np.random.normal(0, 0.5, size=len(x))
y = y_true + noise

# Visualize
plt.figure(figsize=(10, 5))
plt.scatter(x, y, alpha=0.5, label="Noisy data")
plt.plot(x, y_true, "r--", label="True function", linewidth=2)
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.title("Exponential Decay Data")
plt.grid(True, alpha=0.3)
plt.show()

print(f"True parameters: a={a_true}, b={b_true}, c={c_true}")

True parameters: a=10.0, b=0.5, c=2.0


/tmp/ipykernel_787997/3143946998.py:21: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### 2.2 Define the Model

Define your model as a Python function. **Important**: Use `jax.numpy` (jnp) instead of `numpy` for JAX compatibility!

In [5]:
def exponential_decay(x, a, b, c):
    """Exponential decay model: y = a * exp(-b*x) + c

    Parameters:
        a: Amplitude
        b: Decay rate
        c: Offset
    """
    return a * jnp.exp(-b * x) + c


print("✅ Model defined!")

✅ Model defined!


### 2.3 Fit the Model

Now let's fit the model to our data. NLSQ's API is compatible with SciPy's `curve_fit`:

In [6]:
# Initial parameter guess
p0 = [8, 0.4, 1]  # Close to true values: [10, 0.5, 2]

# Fit the model
popt, pcov = curve_fit(exponential_decay, x, y, p0=p0)

# Extract fitted parameters
a_fit, b_fit, c_fit = popt

print("Fitted Parameters:")
print(f"  a = {a_fit:.4f} (true: {a_true})")
print(f"  b = {b_fit:.4f} (true: {b_true})")
print(f"  c = {c_fit:.4f} (true: {c_true})")
print("\n✅ Fitting successful!")

Starting curve fit | {'n_params': 3, 'n_data_points': 100, 'method': 'trf', 'solver': 'auto', 'batch_size': None, 'has_bounds': False, 'dynamic_sizing': False}


Starting least squares optimization | {'method': 'trf', 'n_params': 3, 'loss': 'linear', 'ftol': 1e-08, 'xtol': 1e-08, 'gtol': 1e-08}


Starting TRF optimization (no bounds) | {'n_params': 3, 'n_residuals': 100, 'max_nfev': None}


Optimization: iter=0 | cost=7.279370e+01 | ‖∇f‖=3.984604e+02 | nfev=1


Optimization: iter=1 | cost=1.015358e+01 | ‖∇f‖=5.266063e+00 | step=8.072174e+00 | nfev=2


...

Timer: optimization took 1.096608s


Convergence: reason=`ftol` termination condition is satisfied. | iterations=4 | final_cost=9.682819e+00 | time=1.097s | final_gradient_norm=3.219102971210863e-05


Timer: curve_fit took 1.559108s


Curve fit completed | {'total_time': 1.559107634006068, 'final_cost': 19.365638391939743, 'covariance_warning': False}


Fitted Parameters:
  a = 10.2597 (true: 10.0)
  b = 0.5493 (true: 0.5)
  c = 2.0721 (true: 2.0)

✅ Fitting successful!


### 2.4 Visualize the Results

In [7]:
# Generate fitted curve
y_fit = exponential_decay(x, *popt)

# Plot
plt.figure(figsize=(12, 5))

# Left: Data and fit
plt.subplot(1, 2, 1)
plt.scatter(x, y, alpha=0.5, label="Data")
plt.plot(x, y_true, "r--", label="True", linewidth=2)
plt.plot(x, y_fit, "g-", label="Fitted", linewidth=2)
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.title("Curve Fit Results")
plt.grid(True, alpha=0.3)

# Right: Residuals
plt.subplot(1, 2, 2)
residuals = y - y_fit
plt.scatter(x, residuals, alpha=0.5)
plt.axhline(0, color="r", linestyle="--", linewidth=2)
plt.xlabel("x")
plt.ylabel("Residuals (y - y_fit)")
plt.title("Residual Plot")
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print goodness of fit
rmse = np.sqrt(np.mean(residuals**2))
print(f"RMSE: {rmse:.4f}")

RMSE: 0.4401


/tmp/ipykernel_787997/3459840067.py:29: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### 🎯 Exercise 1: Try It Yourself!

Modify the code above to fit a **linear** model: $y = a \cdot x + b$

**Hints**:
1. Generate linear data: `y_true = 2*x + 1`
2. Define model: `def linear(x, a, b): return a*x + b`
3. Use `p0=[1, 1]` (2 parameters)

<details>
<summary>Click to see solution</summary>

```python
# Generate linear data
x = np.linspace(0, 10, 50)
y = 2*x + 1 + np.random.normal(0, 1, size=len(x))

# Define model
def linear(x, a, b):
    return a*x + b

# Fit
popt, pcov = curve_fit(linear, x, y, p0=[1, 1])
print(f"Fitted: a={popt[0]:.2f}, b={popt[1]:.2f}")
```
</details>

In [8]:
# Your code here


---

## Section 3: Common Fitting Patterns

NLSQ includes a library of common functions for quick fitting.

### 3.1 Using Built-in Functions

In [9]:
# List available functions
print("Available functions in nlsq.functions:")
for func_name in functions.__all__:
    print(f"  - {func_name}")

Available functions in nlsq.functions:
  - exponential_decay
  - exponential_growth
  - gaussian
  - linear
  - polynomial
  - power_law
  - sigmoid


### 3.2 Example: Gaussian Peak Fitting

Fit a Gaussian peak: $y = a \cdot e^{-(x-\mu)^2 / (2\sigma^2)}$

In [10]:
# Generate Gaussian data
x = np.linspace(-5, 5, 100)
a_true, mu_true, sigma_true = 10, 0, 1.5
y_true = a_true * np.exp(-((x - mu_true) ** 2) / (2 * sigma_true**2))
y = y_true + np.random.normal(0, 0.5, size=len(x))

# Fit using built-in gaussian function
from nlsq.functions import gaussian

popt, pcov = curve_fit(gaussian, x, y, p0=[10, 0, 1])
a_fit, mu_fit, sigma_fit = popt

# Plot
plt.figure(figsize=(10, 5))
plt.scatter(x, y, alpha=0.5, label="Data")
plt.plot(x, y_true, "r--", label="True", linewidth=2)
plt.plot(x, gaussian(x, *popt), "g-", label="Fitted", linewidth=2)
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.title(f"Gaussian Fit: μ={mu_fit:.2f}, σ={sigma_fit:.2f}")
plt.grid(True, alpha=0.3)
plt.show()

print(f"Fitted: amplitude={a_fit:.2f}, mean={mu_fit:.2f}, std={sigma_fit:.2f}")

Starting curve fit | {'n_params': 3, 'n_data_points': 100, 'method': 'trf', 'solver': 'auto', 'batch_size': None, 'has_bounds': False, 'dynamic_sizing': False}


Starting least squares optimization | {'method': 'trf', 'n_params': 3, 'loss': 'linear', 'ftol': 1e-08, 'xtol': 1e-08, 'gtol': 1e-08}


Starting TRF optimization (no bounds) | {'n_params': 3, 'n_residuals': 100, 'max_nfev': None}


Optimization: iter=0 | cost=1.442464e+02 | ‖∇f‖=5.587512e+02 | nfev=1


Optimization: iter=1 | cost=1.776593e+01 | ‖∇f‖=8.716723e+01 | step=1.004988e+01 | nfev=2


...

Convergence: reason=`ftol` termination condition is satisfied. | iterations=5 | final_cost=1.103323e+01 | time=0.275s | final_gradient_norm=1.4500226525743232e-06


Timer: curve_fit took 0.636394s


Curve fit completed | {'total_time': 0.6363940360024571, 'final_cost': 22.066462728633894, 'covariance_warning': False}


Fitted: amplitude=9.99, mean=0.02, std=1.51


/tmp/ipykernel_787997/1482938768.py:23: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### 3.3 Example: Sigmoid (Logistic) Curve

Common in dose-response and growth curves: $y = \frac{L}{1 + e^{-k(x-x_0)}} + b$

In [11]:
# Generate sigmoid data
x = np.linspace(0, 10, 100)
L_true, x0_true, k_true, b_true = 10, 5, 1, 0
y_true = L_true / (1 + np.exp(-k_true * (x - x0_true))) + b_true
y = y_true + np.random.normal(0, 0.5, size=len(x))

# Fit using built-in sigmoid function
from nlsq.functions import sigmoid

popt, pcov = curve_fit(sigmoid, x, y, p0=[10, 5, 1, 0])
L_fit, x0_fit, k_fit, b_fit = popt

# Plot
plt.figure(figsize=(10, 5))
plt.scatter(x, y, alpha=0.5, label="Data")
plt.plot(x, y_true, "r--", label="True", linewidth=2)
plt.plot(x, sigmoid(x, *popt), "g-", label="Fitted", linewidth=2)
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.title(f"Sigmoid Fit: L={L_fit:.2f}, x₀={x0_fit:.2f}, k={k_fit:.2f}, b={b_fit:.2f}")
plt.grid(True, alpha=0.3)
plt.show()

print(
    f"Fitted: max={L_fit:.2f}, midpoint={x0_fit:.2f}, steepness={k_fit:.2f}, baseline={b_fit:.2f}"
)

Starting curve fit | {'n_params': 4, 'n_data_points': 100, 'method': 'trf', 'solver': 'auto', 'batch_size': None, 'has_bounds': False, 'dynamic_sizing': False}


Starting least squares optimization | {'method': 'trf', 'n_params': 4, 'loss': 'linear', 'ftol': 1e-08, 'xtol': 1e-08, 'gtol': 1e-08}


Starting TRF optimization (no bounds) | {'n_params': 4, 'n_residuals': 100, 'max_nfev': None}


Optimization: iter=0 | cost=1.460155e+01 | ‖∇f‖=5.241212e+00 | nfev=1


Optimization: iter=1 | cost=1.426073e+01 | ‖∇f‖=1.224733e-01 | step=1.122497e+01 | nfev=2


...

Convergence: reason=`ftol` termination condition is satisfied. | iterations=4 | final_cost=1.426017e+01 | time=0.694s | final_gradient_norm=8.377644655723979e-05


Timer: curve_fit took 1.041609s


Curve fit completed | {'total_time': 1.0416085419710726, 'final_cost': 28.520347977148518, 'covariance_warning': False}


Fitted: max=9.88, midpoint=5.07, steepness=1.00, baseline=0.16


/tmp/ipykernel_787997/3441078113.py:23: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### 3.4 Example: Power Law

Common in scaling relationships: $y = a \cdot x^b$

In [12]:
# Generate power law data
x = np.linspace(1, 10, 50)  # Start from 1 to avoid x=0
a_true, b_true = 2, 1.5
y_true = a_true * x**b_true
y = y_true + np.random.normal(0, 2, size=len(x))

# Fit using built-in power_law function
from nlsq.functions import power_law

popt, pcov = curve_fit(power_law, x, y, p0=[2, 1.5])
a_fit, b_fit = popt

# Plot (log-log scale shows linearity)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Linear scale
ax1.scatter(x, y, alpha=0.5, label="Data")
ax1.plot(x, y_true, "r--", label="True", linewidth=2)
ax1.plot(x, power_law(x, *popt), "g-", label="Fitted", linewidth=2)
ax1.set_xlabel("x")
ax1.set_ylabel("y")
ax1.legend()
ax1.set_title("Power Law (Linear Scale)")
ax1.grid(True, alpha=0.3)

# Log-log scale
ax2.scatter(x, y, alpha=0.5, label="Data")
ax2.plot(x, y_true, "r--", label="True", linewidth=2)
ax2.plot(x, power_law(x, *popt), "g-", label="Fitted", linewidth=2)
ax2.set_xscale("log")
ax2.set_yscale("log")
ax2.set_xlabel("x (log scale)")
ax2.set_ylabel("y (log scale)")
ax2.legend()
ax2.set_title("Power Law (Log-Log Scale)")
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Fitted: y = {a_fit:.2f} * x^{b_fit:.2f}")

Starting curve fit | {'n_params': 2, 'n_data_points': 50, 'method': 'trf', 'solver': 'auto', 'batch_size': None, 'has_bounds': False, 'dynamic_sizing': False}


Starting least squares optimization | {'method': 'trf', 'n_params': 2, 'loss': 'linear', 'ftol': 1e-08, 'xtol': 1e-08, 'gtol': 1e-08}


Starting TRF optimization (no bounds) | {'n_params': 2, 'n_residuals': 50, 'max_nfev': None}


Optimization: iter=0 | cost=5.877460e+01 | ‖∇f‖=1.127077e+03 | nfev=1


Optimization: iter=1 | cost=5.024109e+01 | ‖∇f‖=1.013755e+03 | step=2.500000e+00 | nfev=2


...

Convergence: reason=`ftol` termination condition is satisfied. | iterations=4 | final_cost=4.802030e+01 | time=0.625s | final_gradient_norm=2.2010425482221763e-08


Timer: curve_fit took 1.016214s


Curve fit completed | {'total_time': 1.0162138800369576, 'final_cost': 96.04060228539663, 'covariance_warning': False}


Fitted: y = 2.30 * x^1.43


/tmp/ipykernel_787997/466641944.py:39: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### 3.5 Example: Polynomial Fitting

Fit polynomials of any degree: $y = a_0 + a_1x + a_2x^2 + ...$

In [13]:
# Generate polynomial data (degree 3)
x = np.linspace(-3, 3, 100)
coeffs_true = [0.5, -2, 1, 3]  # y = 0.5x³ - 2x² + x + 3
y_true = np.polyval(coeffs_true, x)
y = y_true + np.random.normal(0, 1, size=len(x))

# Fit using built-in polynomial function
from nlsq.functions import polynomial

# Create polynomial function for degree 3
poly3 = polynomial(3)

popt, pcov = curve_fit(poly3, x, y, p0=[1, -1, 1, 1])

# Plot
plt.figure(figsize=(10, 5))
plt.scatter(x, y, alpha=0.5, label="Data")
plt.plot(x, y_true, "r--", label="True", linewidth=2)
plt.plot(x, poly3(x, *popt), "g-", label="Fitted", linewidth=2)
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.title("Polynomial Fit (Degree 3)")
plt.grid(True, alpha=0.3)
plt.show()

print(f"Fitted coefficients: {popt}")
print(f"True coefficients:   {coeffs_true}")

Starting curve fit | {'n_params': 4, 'n_data_points': 100, 'method': 'trf', 'solver': 'auto', 'batch_size': None, 'has_bounds': False, 'dynamic_sizing': False}


Starting least squares optimization | {'method': 'trf', 'n_params': 4, 'loss': 'linear', 'ftol': 1e-08, 'xtol': 1e-08, 'gtol': 1e-08}


Starting TRF optimization (no bounds) | {'n_params': 4, 'n_residuals': 100, 'max_nfev': None}


Optimization: iter=0 | cost=1.974571e+03 | ‖∇f‖=5.716990e+03 | nfev=1


Optimization: iter=1 | cost=5.523791e+01 | ‖∇f‖=1.890068e+01 | step=4.000000e+00 | nfev=2


...

Convergence: reason=`gtol` termination condition is satisfied. | iterations=2 | final_cost=5.030355e+01 | time=0.661s | final_gradient_norm=2.7622348852673895e-13


Timer: curve_fit took 0.930325s


Curve fit completed | {'total_time': 0.9303253329126164, 'final_cost': 100.60709662912501, 'covariance_warning': False}


Fitted coefficients: [ 0.46764853 -2.02846206  1.09938547  3.1534885 ]
True coefficients:   [0.5, -2, 1, 3]


/tmp/ipykernel_787997/4158310458.py:25: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### 📋 Summary: Common Functions

| Function | Equation | Use Cases |
|----------|----------|----------|
| `linear` | $y = ax + b$ | Calibration, trends |
| `exponential_decay` | $y = ae^{-bx} + c$ | Radioactive decay, RC circuits |
| `exponential_growth` | $y = ae^{bx} + c$ | Population growth, interest |
| `gaussian` | $y = ae^{-(x-\mu)^2/(2\sigma^2)}$ | Spectroscopy, normal distributions |
| `sigmoid` | $y = L/(1+e^{-k(x-x_0)})$ | Dose-response, growth curves |
| `power_law` | $y = ax^b$ | Scaling laws, allometry |
| `polynomial` | $y = \sum a_i x^i$ | Flexible curve fitting |

**Next**: Learn how to handle bounds and constraints!

---

## Section 4: Parameter Bounds and Constraints

Real-world problems often require parameter constraints. NLSQ supports bounds like SciPy.

### 4.1 Fitting with Bounds

Let's fit an exponential with constrained parameters:

In [14]:
# Generate data with known parameters
x = np.linspace(0, 10, 100)
a_true, b_true, c_true = 5.0, 0.3, 1.0
y = a_true * np.exp(-b_true * x) + c_true + np.random.normal(0, 0.2, size=len(x))

# Define bounds: (lower, upper) for each parameter
# bounds = ([a_min, b_min, c_min], [a_max, b_max, c_max])
bounds = ([0, 0, 0], [10, 1, 5])  # All parameters must be positive

# Fit with bounds
popt, pcov = curve_fit(exponential_decay, x, y, p0=[1, 0.1, 0.5], bounds=bounds)

print("Fitted with bounds:")
print(f"  a = {popt[0]:.4f} (true: {a_true}, bounds: 0-10)")
print(f"  b = {popt[1]:.4f} (true: {b_true}, bounds: 0-1)")
print(f"  c = {popt[2]:.4f} (true: {c_true}, bounds: 0-5)")

# Visualize
plt.figure(figsize=(10, 5))
plt.scatter(x, y, alpha=0.5, label="Data")
plt.plot(x, exponential_decay(x, *popt), "g-", label="Fitted (bounded)", linewidth=2)
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.title("Curve Fit with Parameter Bounds")
plt.grid(True, alpha=0.3)
plt.show()

Starting curve fit | {'n_params': 3, 'n_data_points': 100, 'method': 'trf', 'solver': 'auto', 'batch_size': None, 'has_bounds': True, 'dynamic_sizing': False}


Starting least squares optimization | {'method': 'trf', 'n_params': 3, 'loss': 'linear', 'ftol': 1e-08, 'xtol': 1e-08, 'gtol': 1e-08}


Timer: optimization took 1.215584s


Convergence: reason=`ftol` termination condition is satisfied. | iterations=8 | final_cost=1.906042e+00 | time=1.216s | final_gradient_norm=2.626256760029409e-07


Timer: curve_fit took 1.478551s


Curve fit completed | {'total_time': 1.4785506250336766, 'final_cost': 3.812083022924312, 'covariance_warning': False}


Fitted with bounds:
  a = 5.0692 (true: 5.0, bounds: 0-10)
  b = 0.2858 (true: 0.3, bounds: 0-1)
  c = 0.8800 (true: 1.0, bounds: 0-5)


/tmp/ipykernel_787997/4022721614.py:27: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### 4.2 Parameter Uncertainties

The covariance matrix `pcov` provides parameter uncertainties:

In [15]:
# Extract standard errors from covariance matrix
perr = np.sqrt(np.diag(pcov))

print("Parameter uncertainties (1σ):")
print(f"  a = {popt[0]:.4f} ± {perr[0]:.4f}")
print(f"  b = {popt[1]:.4f} ± {perr[1]:.4f}")
print(f"  c = {popt[2]:.4f} ± {perr[2]:.4f}")

# Calculate correlation matrix
corr = pcov / np.outer(perr, perr)

print("\nParameter correlation matrix:")
print(corr)

Parameter uncertainties (1σ):
  a = 5.0692 ± 0.0784
  b = 0.2858 ± 0.0137
  c = 0.8800 ± 0.0789

Parameter correlation matrix:
[[ 1.         -0.23246237 -0.52436989]
 [-0.23246237  1.          0.91313873]
 [-0.52436989  0.91313873  1.        ]]


---

## Section 5: Error Handling and Diagnostics

NLSQ provides helpful error messages and diagnostics when fits fail.

### 5.1 Common Issues and Solutions

In [16]:
# Example 1: Bad initial guess
print("Example 1: Bad initial guess")
print("=" * 50)
try:
    # p0 too far from true values
    popt, pcov = curve_fit(exponential_decay, x, y, p0=[100, 10, 50])
    print("✅ Fit succeeded despite bad p0!")
    print(f"Fitted: a={popt[0]:.2f}, b={popt[1]:.2f}, c={popt[2]:.2f}")
except Exception as e:
    print(f"❌ Fit failed: {e}")
    print("💡 Tip: Try better p0 estimates or increase max_nfev")

# Example 2: Conflicting bounds
print("\nExample 2: Conflicting bounds")
print("=" * 50)
try:
    # p0 outside bounds
    bad_bounds = ([0, 0, 0], [1, 0.1, 0.5])  # Too restrictive
    popt, pcov = curve_fit(exponential_decay, x, y, p0=[5, 0.3, 1], bounds=bad_bounds)
except Exception as e:
    print(f"❌ Fit failed: {type(e).__name__}")
    print("💡 Tip: Ensure p0 is within bounds, and bounds are reasonable")

# Example 3: Successful fit with diagnostics
print("\nExample 3: Successful fit with full diagnostics")
print("=" * 50)
popt, pcov = curve_fit(exponential_decay, x, y, p0=[5, 0.3, 1], full_output=False)
print("✅ Fit succeeded!")
print(f"Final parameters: a={popt[0]:.4f}, b={popt[1]:.4f}, c={popt[2]:.4f}")
print(f"Condition number of covariance: {np.linalg.cond(pcov):.2e}")
if np.linalg.cond(pcov) > 1e10:
    print("⚠️  Warning: Poorly conditioned covariance (parameters may be correlated)")

Starting curve fit | {'n_params': 3, 'n_data_points': 100, 'method': 'trf', 'solver': 'auto', 'batch_size': None, 'has_bounds': False, 'dynamic_sizing': False}


Starting least squares optimization | {'method': 'trf', 'n_params': 3, 'loss': 'linear', 'ftol': 1e-08, 'xtol': 1e-08, 'gtol': 1e-08}


Example 1: Bad initial guess


Starting TRF optimization (no bounds) | {'n_params': 3, 'n_residuals': 100, 'max_nfev': None}


Optimization: iter=0 | cost=1.253015e+05 | ‖∇f‖=4.901075e+03 | nfev=1


...

Timer: optimization took 0.313842s


Convergence: reason=`ftol` termination condition is satisfied. | iterations=3 | final_cost=1.906042e+00 | time=0.314s | final_gradient_norm=3.744730355248471e-06


Timer: curve_fit took 0.547602s


Curve fit completed | {'total_time': 0.5476020369678736, 'final_cost': 3.812083022924377, 'covariance_warning': False}


✅ Fit succeeded!
Final parameters: a=5.0692, b=0.2858, c=0.8800
Condition number of covariance: 6.34e+02


### 5.2 Monitoring Progress with Callbacks

Use callbacks to monitor optimization progress:

In [17]:
# Create a progress callback
iteration_data = []


def progress_callback(iteration, cost, params, info=None):
    iteration_data.append(
        {"iter": iteration, "cost": cost, "params": params.copy(), "info": info}
    )
    if iteration % 5 == 0:  # Print every 5 iterations
        print(f"Iteration {iteration:3d}: cost = {cost:.6e}, params = {params}")


# Fit with callback
print("Fitting with progress monitoring:")
print("=" * 70)
popt, pcov = curve_fit(
    exponential_decay, x, y, p0=[1, 0.1, 0.5], callback=progress_callback
)

print("\n✅ Optimization complete!")
print(f"Total iterations: {len(iteration_data)}")
if len(iteration_data) > 0:
    print(f"Final cost: {iteration_data[-1]['cost']:.6e}")

# Plot convergence
if len(iteration_data) > 0:
    costs = [d["cost"] for d in iteration_data]
    plt.figure(figsize=(10, 5))
    plt.semilogy(costs)
    plt.xlabel("Iteration")
    plt.ylabel("Cost (log scale)")
    plt.title("Optimization Convergence")
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    print("No iteration data collected (optimization converged immediately)")

Starting curve fit | {'n_params': 3, 'n_data_points': 100, 'method': 'trf', 'solver': 'auto', 'batch_size': None, 'has_bounds': False, 'dynamic_sizing': False}


Starting least squares optimization | {'method': 'trf', 'n_params': 3, 'loss': 'linear', 'ftol': 1e-08, 'xtol': 1e-08, 'gtol': 1e-08}


Fitting with progress monitoring:


Starting TRF optimization (no bounds) | {'n_params': 3, 'n_residuals': 100, 'max_nfev': None}


Optimization: iter=0 | cost=1.690529e+02 | ‖∇f‖=2.554469e+02 | nfev=1


...

Convergence: reason=`ftol` termination condition is satisfied. | iterations=8 | final_cost=1.906042e+00 | time=0.332s | final_gradient_norm=5.550148821920065e-07


Timer: curve_fit took 0.598971s


Curve fit completed | {'total_time': 0.5989708270644769, 'final_cost': 3.8120830229243143, 'covariance_warning': False}


Iteration   5: cost = 1.992400e+00, params = [5.0522334  0.27914186 0.89301235]

✅ Optimization complete!
Total iterations: 8
Final cost: 1.906042e+00


/tmp/ipykernel_787997/517946241.py:34: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


---

## Section 6: Large Dataset Handling

NLSQ can handle millions of points efficiently, especially on GPU.

### 6.1 Fitting Large Datasets

In [18]:
import time

# Generate large dataset
n_points = 100000  # 100K points
x_large = np.linspace(0, 10, n_points)
y_large = 5.0 * np.exp(-0.3 * x_large) + 1.0 + np.random.normal(0, 0.2, size=n_points)

print(f"Dataset size: {n_points:,} points")
print(f"Memory: ~{(x_large.nbytes + y_large.nbytes) / 1024**2:.2f} MB")

# Fit large dataset
print("\nFitting large dataset...")
start = time.time()
popt_large, pcov_large = curve_fit(exponential_decay, x_large, y_large, p0=[5, 0.3, 1])
elapsed = time.time() - start

print(f"✅ Fit complete in {elapsed:.3f} seconds")
print(f"Fitted: a={popt_large[0]:.4f}, b={popt_large[1]:.4f}, c={popt_large[2]:.4f}")
print(f"Processing rate: {n_points / elapsed:,.0f} points/second")

# Note: First run includes JIT compilation overhead
print("\n💡 Note: First fit includes JAX JIT compilation (~1-2 seconds).")
print("   Subsequent fits reuse compiled code and are much faster!")

Starting curve fit | {'n_params': 3, 'n_data_points': 100000, 'method': 'trf', 'solver': 'auto', 'batch_size': None, 'has_bounds': False, 'dynamic_sizing': False}


Starting least squares optimization | {'method': 'trf', 'n_params': 3, 'loss': 'linear', 'ftol': 1e-08, 'xtol': 1e-08, 'gtol': 1e-08}


Dataset size: 100,000 points
Memory: ~1.53 MB

Fitting large dataset...


Starting TRF optimization (no bounds) | {'n_params': 3, 'n_residuals': 100000, 'max_nfev': None}


Optimization: iter=0 | cost=2.004095e+03 | ‖∇f‖=6.686086e+01 | nfev=1


...

Timer: optimization took 0.352300s


Convergence: reason=`ftol` termination condition is satisfied. | iterations=2 | final_cost=2.004091e+03 | time=0.352s | final_gradient_norm=2.0421479220544825e-05


Timer: curve_fit took 0.830022s


Curve fit completed | {'total_time': 0.830022380920127, 'final_cost': 4008.1829301685275, 'covariance_warning': False}


✅ Fit complete in 0.933 seconds
Fitted: a=4.9999, b=0.3001, c=1.0007
Processing rate: 107,234 points/second

💡 Note: First fit includes JAX JIT compilation (~1-2 seconds).
   Subsequent fits reuse compiled code and are much faster!


### 6.2 Automatic Chunking for Very Large Datasets

For datasets larger than available memory, NLSQ can automatically chunk:

In [19]:
from nlsq import curve_fit_large

# Simulate very large dataset (10M points would be ~160 MB)
# Using 500K points for demo (faster in Colab)
n_huge = 500000
x_huge = np.linspace(0, 10, n_huge)
y_huge = 5.0 * np.exp(-0.3 * x_huge) + 1.0 + np.random.normal(0, 0.2, size=n_huge)

print(
    f"Dataset size: {n_huge:,} points ({(x_huge.nbytes + y_huge.nbytes) / 1024**2:.1f} MB)"
)

# Fit with automatic chunking
print("\nFitting with automatic chunking...")
start = time.time()
popt_huge, pcov_huge = curve_fit_large(
    exponential_decay,
    x_huge,
    y_huge,
    p0=[5, 0.3, 1],
    chunk_size=50000,  # Process 50K points at a time
)
elapsed = time.time() - start

print(f"✅ Fit complete in {elapsed:.3f} seconds")
print(f"Fitted: a={popt_huge[0]:.4f}, b={popt_huge[1]:.4f}, c={popt_huge[2]:.4f}")
print(f"Processing rate: {n_huge / elapsed:,.0f} points/second")
print("\n💡 curve_fit_large() automatically manages memory for huge datasets!")

Starting curve fit | {'n_params': 3, 'n_data_points': 500000, 'method': 'trf', 'solver': 'auto', 'batch_size': None, 'has_bounds': False, 'dynamic_sizing': False}


Starting least squares optimization | {'method': 'trf', 'n_params': 3, 'loss': 'linear', 'ftol': 1e-08, 'xtol': 1e-08, 'gtol': 1e-08}


Dataset size: 500,000 points (7.6 MB)

Fitting with automatic chunking...


Starting TRF optimization (no bounds) | {'n_params': 3, 'n_residuals': 500000, 'max_nfev': None}


Optimization: iter=0 | cost=1.001438e+04 | ‖∇f‖=9.200474e+02 | nfev=1


...

Timer: optimization took 0.400037s


Convergence: reason=`ftol` termination condition is satisfied. | iterations=2 | final_cost=1.001432e+04 | time=0.400s | final_gradient_norm=4.113231852898025e-07


Timer: curve_fit took 0.778378s


Curve fit completed | {'total_time': 0.7783778180601075, 'final_cost': 20028.638064800434, 'covariance_warning': False}


✅ Fit complete in 0.866 seconds
Fitted: a=5.0013, b=0.3001, c=0.9995
Processing rate: 577,141 points/second

💡 curve_fit_large() automatically manages memory for huge datasets!


---

## Section 7: GPU Acceleration

NLSQ automatically uses GPU when available. Let's benchmark CPU vs GPU performance.

### 7.1 GPU Performance Comparison

In [20]:
import jax

# Check current backend
current_backend = jax.devices()[0].platform
print(f"Current backend: {current_backend}")

if current_backend == "gpu":
    print("\n🚀 GPU detected! Running performance comparison...")

    # Create large dataset for GPU test
    n_gpu = 1000000  # 1M points
    x_gpu = np.linspace(0, 10, n_gpu)
    y_gpu = 5.0 * np.exp(-0.3 * x_gpu) + 1.0 + np.random.normal(0, 0.2, size=n_gpu)

    # Warmup (JIT compilation)
    print("Warming up JIT compiler...")
    _ = curve_fit(exponential_decay, x_gpu[:1000], y_gpu[:1000], p0=[5, 0.3, 1])

    # GPU timing
    print(f"\nFitting {n_gpu:,} points on GPU...")
    start = time.time()
    popt_gpu, _ = curve_fit(exponential_decay, x_gpu, y_gpu, p0=[5, 0.3, 1])
    gpu_time = time.time() - start

    print(f"✅ GPU fit: {gpu_time:.3f} seconds")
    print(f"   Processing rate: {n_gpu / gpu_time:,.0f} points/second")
    print(
        f"   Parameters: a={popt_gpu[0]:.4f}, b={popt_gpu[1]:.4f}, c={popt_gpu[2]:.4f}"
    )

    # Note: CPU comparison would require JAX_PLATFORM_NAME=cpu environment variable
    print("\n💡 GPU acceleration provides 100-300x speedup vs SciPy on large datasets!")
else:
    print("\n💻 Running on CPU. To use GPU:")
    print("   1. Go to Runtime → Change runtime type")
    print("   2. Select GPU as hardware accelerator")
    print("   3. Restart runtime and re-run notebook")

Starting curve fit | {'n_params': 3, 'n_data_points': 1000, 'method': 'trf', 'solver': 'auto', 'batch_size': None, 'has_bounds': False, 'dynamic_sizing': False}


Starting least squares optimization | {'method': 'trf', 'n_params': 3, 'loss': 'linear', 'ftol': 1e-08, 'xtol': 1e-08, 'gtol': 1e-08}


Current backend: gpu

🚀 GPU detected! Running performance comparison...
Warming up JIT compiler...


Starting TRF optimization (no bounds) | {'n_params': 3, 'n_residuals': 1000, 'max_nfev': None}


Optimization: iter=0 | cost=1.921085e+01 | ‖∇f‖=9.869230e+00 | nfev=1


...

Timer: optimization took 0.422347s


Convergence: reason=`ftol` termination condition is satisfied. | iterations=2 | final_cost=2.004007e+04 | time=0.422s | final_gradient_norm=2.3727335729972765e-05


Timer: curve_fit took 0.930436s


Curve fit completed | {'total_time': 0.9304362749680877, 'final_cost': 40080.14021425297, 'covariance_warning': False}


✅ GPU fit: 1.024 seconds
   Processing rate: 976,476 points/second
   Parameters: a=5.0005, b=0.3003, c=1.0009

💡 GPU acceleration provides 100-300x speedup vs SciPy on large datasets!


### 7.2 Advanced Features

NLSQ includes several advanced features:

In [21]:
# 1. Automatic p0 estimation
print("1. Automatic p0 estimation")
print("=" * 50)

# Generate data
x = np.linspace(0, 10, 100)
y = 5.0 * np.exp(-0.3 * x) + 1.0 + np.random.normal(0, 0.2, size=len(x))

# Auto-estimate p0 using function method
from nlsq.functions import exponential_decay

p0_auto = exponential_decay.estimate_p0(x, y)
print(f"Auto-estimated p0: {p0_auto}")
popt, _ = curve_fit(exponential_decay, x, y, p0=p0_auto)
print("✅ Fit succeeded with auto p0!")
print(f"Fitted: a={popt[0]:.4f}, b={popt[1]:.4f}, c={popt[2]:.4f}")
print("True values: a=5.0000, b=0.3000, c=1.0000")

# 2. Robust fitting (loss functions)
print("\n2. Robust fitting with loss functions")
print("=" * 50)

# Add outliers to data
y_outliers = y.copy()
outlier_indices = np.random.choice(len(y), 10, replace=False)
y_outliers[outlier_indices] += np.random.normal(0, 5, size=10)

# Standard fit (sensitive to outliers)
popt_standard, _ = curve_fit(exponential_decay, x, y_outliers, p0=[5, 0.3, 1])

# Robust fit with Huber loss
popt_robust, _ = curve_fit(
    exponential_decay,
    x,
    y_outliers,
    p0=[5, 0.3, 1],
    loss="huber",  # Robust to outliers
)

print(
    f"Standard fit: a={popt_standard[0]:.4f}, b={popt_standard[1]:.4f}, c={popt_standard[2]:.4f}"
)
print(
    f"Robust fit:   a={popt_robust[0]:.4f}, b={popt_robust[1]:.4f}, c={popt_robust[2]:.4f}"
)
print("True values:  a=5.0000, b=0.3000, c=1.0000")
print("\n✅ Robust fit is closer to true values despite outliers!")

# Visualize
plt.figure(figsize=(12, 5))
plt.scatter(x, y_outliers, alpha=0.5, label="Data with outliers", color="gray")
plt.scatter(
    x[outlier_indices],
    y_outliers[outlier_indices],
    color="red",
    s=100,
    marker="x",
    linewidth=3,
    label="Outliers",
)
plt.plot(
    x, exponential_decay(x, *popt_standard), "b--", label="Standard fit", linewidth=2
)
plt.plot(
    x,
    exponential_decay(x, *popt_robust),
    "g-",
    label="Robust fit (Huber loss)",
    linewidth=2,
)
plt.plot(x, exponential_decay(x, 5, 0.3, 1), "r:", label="True function", linewidth=2)
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.title("Robust Fitting with Huber Loss")
plt.grid(True, alpha=0.3)
plt.show()

Starting curve fit | {'n_params': 3, 'n_data_points': 100, 'method': 'trf', 'solver': 'auto', 'batch_size': None, 'has_bounds': False, 'dynamic_sizing': False}


Starting least squares optimization | {'method': 'trf', 'n_params': 3, 'loss': 'linear', 'ftol': 1e-08, 'xtol': 1e-08, 'gtol': 1e-08}


1. Automatic p0 estimation
Auto-estimated p0: [5.117433381931369, 0.24507703884083779, 0.8608223294343962]


Starting TRF optimization (no bounds) | {'n_params': 3, 'n_residuals': 100, 'max_nfev': None}


Optimization: iter=0 | cost=3.749386e+00 | ‖∇f‖=1.177043e+02 | nfev=1


...

Convergence: reason=`ftol` termination condition is satisfied. | iterations=3 | final_cost=4.794278e+01 | time=0.509s | final_gradient_norm=0.0002109366763817233


Timer: curve_fit took 1.046079s


Curve fit completed | {'total_time': 1.0460794219980016, 'final_cost': 95.88555456256519, 'covariance_warning': False}


Standard fit: a=7.9643, b=0.1181, c=-2.4349
Robust fit:   a=5.0059, b=0.2655, c=0.7825
True values:  a=5.0000, b=0.3000, c=1.0000

✅ Robust fit is closer to true values despite outliers!


/tmp/ipykernel_787997/2188551818.py:77: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


---

## Section 8: Conclusion and Next Steps

### 🎉 Congratulations!

You've completed the NLSQ interactive tutorial! You now know how to:

✅ Install and set up NLSQ with GPU support  
✅ Fit curves with common models (exponential, Gaussian, sigmoid, power law)  
✅ Apply parameter bounds and constraints  
✅ Handle errors and monitor optimization progress  
✅ Work with large datasets (millions of points)  
✅ Leverage GPU acceleration for 100x+ speedups  
✅ Use advanced features (callbacks, robust fitting)

### 📚 Additional Resources

- **Documentation**: https://nlsq.readthedocs.io
- **GitHub**: https://github.com/imewei/NLSQ
- **Examples**: Browse the `examples/` directory for more use cases
- **API Reference**: Complete function documentation and parameters

### 🚀 Next Steps

1. **Try your own data**: Replace the example data with your real-world datasets
2. **Explore loss functions**: Try different loss functions for robust fitting
3. **Batch processing**: Fit multiple curves in parallel for maximum efficiency
4. **Custom models**: Define your own fitting functions with JAX
5. **Performance tuning**: Experiment with different algorithms and tolerances

### 💬 Get Help

- **Issues**: Report bugs at https://github.com/imewei/NLSQ/issues
- **Discussions**: Ask questions in GitHub Discussions
- **Citation**: If you use NLSQ in research, please cite the original JAXFit paper

### 🙏 Acknowledgments

NLSQ is based on JAXFit by Lucas R. Hofer, Milan Krstajić, and Robert P. Smith.  
Development supported by Argonne National Laboratory.

---

**Happy fitting! 🎯**